In [6]:
#installing necessary packages

!pip install pandas
!pip install keras==2.3.1
!pip install sklearn
!pip install statsmodels
!pip install warnings

In [371]:
#importing necessary packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [378]:
data = pd.read_csv('clean_data.csv', header=0, index_col=0)

In [379]:
data.shape

(1821, 137)

In [46]:
data.head()

,eur_jpy,nikkei_Open,dow_jones_Open,dax_Open,dax_Volume,ftse_Open,euronext_Open,eurodollar_Price,eurodollar_Vol_thousands,eurodollar_Change,...,exhibit_force_goldstein,fight_goldstein,intent_to_cooperate_goldstein,material_cooperation_goldstein,protest_goldstein,provide_aid_goldstein,public statement_goldstein,reduce_relations_goldstein,threaten_goldstein,yield_goldstein
datetime,,,,,,,,,,,,,,,,,,,,,
2015-01-01,144.81,17325.68,17823.07,25.85,2700.0,6566.090000,839.52,99.75,0.860000,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02,144.60,17325.68,17823.07,25.85,2700.0,6566.090000,835.97,99.75,0.860000,0.00,...,NaN,-10.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2015-01-03,143.99,17325.68,17822.48,25.85,2700.0,6559.993333,827.91,99.73,5.953333,-0.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2015-01-04,143.38,17325.68,17821.89,25.85,2700.0,6553.896667,819.85,99.71,11.046667,-0.04,...,NaN,NaN,NaN,NaN,NaN,7.4,NaN,NaN,NaN,5.0
2015-01-05,142.77,17325.68,17821.30,25.85,2700.0,6547.800000,811.79,99.75,16.140000,0.04,...,NaN,NaN,NaN,NaN,NaN,7.0,0.0,NaN,NaN,5.0


In [121]:
data.describe()

,eur_jpy,nikkei_Open,dow_jones_Open,dax_Open,dax_Volume,ftse_Open,euronext_Open,eurodollar_Price,eurodollar_Vol_thousands,eurodollar_Change,...,exhibit_force_goldstein,fight_goldstein,intent_to_cooperate_goldstein,material_cooperation_goldstein,protest_goldstein,provide_aid_goldstein,public statement_goldstein,reduce_relations_goldstein,threaten_goldstein,yield_goldstein
count,1821.000000,1821.000000,1821.000000,1821.000000,1821.000000,1821.000000,1821.000000,1821.000000,1821.000000,1821.000000,...,4.400000e+01,410.000000,547.000000,430.000000,64.000000,259.000000,625.000000,40.000000,91.000000,465.000000
mean,126.762158,20064.604519,21734.204647,26.928791,7975.078345,7015.492986,980.179406,98.595043,99.261412,-0.001784,...,-7.200000e+00,-9.949573,4.271136,6.318310,-6.515625,7.234289,0.023027,-4.887500,-5.028571,6.026703
std,6.610295,2180.021214,3723.540904,2.542152,21247.733984,491.243359,81.005648,0.831544,117.900978,0.022462,...,8.984467e-16,0.142149,0.633033,0.264744,0.125000,0.276498,0.415464,1.402779,1.102450,1.371912
min,111.120000,15094.710000,15676.260000,21.290000,9.000000,5536.970000,771.430000,97.180000,0.010000,-0.170000,...,-7.200000e+00,-10.000000,4.000000,6.000000,-7.500000,7.000000,-2.000000,-8.000000,-7.000000,5.000000
25%,121.800000,18604.870000,17977.110000,24.900200,1618.000000,6745.526667,909.690000,97.710000,14.460000,-0.010000,...,-7.200000e+00,-10.000000,4.000000,6.000000,-6.500000,7.000000,0.000000,-7.000000,-5.050000,5.000000
50%,127.720000,20154.380000,21411.190000,26.550000,3047.000000,7138.780000,991.390000,98.690000,43.180000,0.000000,...,-7.200000e+00,-10.000000,4.000000,6.400000,-6.500000,7.266667,0.000000,-4.000000,-4.400000,5.000000
75%,132.060000,21795.610000,25294.970000,28.820000,6349.000000,7386.940000,1044.910000,99.350000,166.150000,0.003333,...,-7.200000e+00,-10.000000,4.000000,6.400000,-6.500000,7.400000,0.000000,-4.000000,-4.400000,7.000000
max,144.810000,24376.170000,28675.340000,33.960000,274734.000000,7877.450000,1156.610000,99.750000,772.820000,0.170000,...,-7.200000e+00,-9.500000,7.000000,7.400000,-6.500000,8.500000,3.400000,-4.000000,-4.400000,10.000000


In [380]:
train, test = data[:1456], data[1456:]

In [306]:
test.head()

,eur_jpy,nikkei_Open,dow_jones_Open,dax_Open,dax_Volume,ftse_Open,euronext_Open,eurodollar_Price,eurodollar_Vol_thousands,eurodollar_Change,...,exhibit_force_goldstein,fight_goldstein,intent_to_cooperate_goldstein,material_cooperation_goldstein,protest_goldstein,provide_aid_goldstein,public statement_goldstein,reduce_relations_goldstein,threaten_goldstein,yield_goldstein
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-01,125.86,19784.880000,23106.275000,23.761,4586.0,6728.130000,916.630000,97.245000,47.705000,0.000000,...,NaN,-10.0,NaN,NaN,NaN,7.4,NaN,NaN,NaN,NaN
2019-01-02,123.57,19741.630000,23058.610000,23.522,3688.0,6728.130000,910.350000,97.250000,50.990000,0.010000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,122.64,19698.380000,23176.390000,23.490,7037.0,6734.230000,897.450000,97.260000,60.220000,0.010000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-04,123.67,19655.130000,22894.920000,23.580,2614.0,6692.660000,921.300000,97.220000,87.420000,-0.040000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2019-01-05,124.02,19751.623333,23088.033333,23.780,2863.0,6740.913333,920.753333,97.223333,82.006667,-0.023333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0


In [381]:
#normalizing data. fit the scaler on the training set and then apply the transform to train & test

scaler = MinMaxScaler()
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [382]:
#replace NaN values in news data with -1. NN will learn that -1 means there is missing data there
#using -1 instead of 0 because 0 is used throughout dataset as valid observation

scaled_train[np.isnan(scaled_train)]=-1
scaled_test[np.isnan(scaled_test)]=-1

In [309]:
#check that all NA values have been converted
print(pd.DataFrame(scaled_train).isna().sum())

0      0
1      0
2      0
3      0
4      0
      ..
132    0
133    0
134    0
135    0
136    0
Length: 137, dtype: int64


In [310]:
#verify that data looks as expected
pd.DataFrame(scaled_train).describe()

,0,1,2,3,4,5,6,7,8,9,...,127,128,129,130,131,132,133,134,135,136
count,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,...,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000
mean,0.499277,0.491497,0.438733,0.460724,0.032101,0.603885,0.597661,0.637606,0.121872,0.491452,...,-0.970467,-0.744609,-0.669869,-0.687955,-0.929258,-0.824820,-0.492955,-0.961195,-0.916209,-0.691083
std,0.201086,0.237422,0.287355,0.217261,0.083498,0.222567,0.234916,0.299415,0.151532,0.065966,...,0.169353,0.478910,0.514691,0.543618,0.368627,0.424512,0.665367,0.268709,0.382861,0.544911
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,0.357896,0.290386,0.192348,0.276243,0.006055,0.455828,0.368979,0.443580,0.014544,0.470588,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,0.543485,0.485743,0.375147,0.442778,0.011802,0.632238,0.631852,0.704280,0.043122,0.500000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
75%,0.640324,0.702955,0.750487,0.625954,0.027141,0.787862,0.802615,0.856031,0.207438,0.509804,...,-1.000000,-1.000000,0.000000,0.000000,-1.000000,-1.000000,0.370370,-1.000000,-1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [412]:
len(scaled_train)

1456

In [416]:
len(generator)

456

In [434]:
#using generator to transform array into time series format for lstm

n_input = 1
n_features = scaled_train.shape[1]

generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)
test_generator = TimeseriesGenerator(scaled_test, scaled_test, length=n_input, batch_size=1)

In [435]:
#confirming the number of samples in my generator

print('Samples: %d' % len(generator))

Samples: 1455


In [436]:
#defining our model

es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=15)

#save the weights from the model with the highest val accuracy
checkpoint = ModelCheckpoint("model_lstm_3.h5", monitor='val_accuracy', verbose=1, save_best_only=True, 
                             save_weights_only=False, mode='auto', period=1)

#changes with the decay rate so that it decays faster
opt = Adam(lr=(1e-4), beta_1 = .85, beta_2 = .99)

model = Sequential()
model.add(LSTM(250, return_sequences=True, input_shape=(n_input, n_features)))

model.add(LSTM(200, return_sequences=True, input_shape=(n_input, n_features)))
model.add(Dropout(.15))

model.add(LSTM(150, input_shape=(n_input, n_features)))
model.add(Dropout(.15))

model.add(Dense(150))
model.add(Dropout(.15))

model.add(Dense(137))

model.compile(loss='mse', optimizer=opt, metrics=['accuracy','mae'])

model.summary()

Model: "sequential_70"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_141 (LSTM)              (None, 1, 250)            388000    
_________________________________________________________________
lstm_142 (LSTM)              (None, 1, 200)            360800    
_________________________________________________________________
dropout_199 (Dropout)        (None, 1, 200)            0         
_________________________________________________________________
lstm_143 (LSTM)              (None, 150)               210600    
_________________________________________________________________
dropout_200 (Dropout)        (None, 150)               0         
_________________________________________________________________
dense_167 (Dense)            (None, 150)               22650     
_________________________________________________________________
dropout_201 (Dropout)        (None, 150)             

In [5]:
#training the model

cb = TensorBoard() # TensorBoard Callback
history_callback = model.fit(generator, epochs=150, validation_data=test_generator, callbacks=[cb, es, checkpoint])
print('Accuracy: '+str(model.evaluate(test_generator, verbose=0)[1]))

In [1]:
#plotting train and test errors

fig, (ax1, ax2) = plt.subplots(2, figsize=(8, 6))


train_acc = history_callback.history['accuracy']
train_loss = history_callback.history['loss']
val_acc = history_callback.history['val_accuracy']
val_loss = history_callback.history['val_loss']
ax1.plot(val_acc, label="test")
ax1.plot(train_acc, label = 'train')
ax2.plot(val_loss, label="test")
ax2.plot(train_loss, label="train")

ax1.set_ylabel('validation accuracy')
ax2.set_ylabel('validation loss')
ax2.set_xlabel('epochs')
ax1.legend()
ax2.legend()
plt.show()

In [2]:
# Calculate the train loss and train metric, in this case mean absolute error
train_loss = np.mean(history_callback.history['loss'])
train_mae = np.mean(history_callback.history['mae'])
score = model.evaluate(test_generator, verbose=0)   


title = 'Train Loss: {0:.3f} Test Loss: {1:.3f}\n  Train MAE: {2:.3f}, Val MAE: {3:.3f}'.format(train_loss, score[0], train_mae, score[1])

# Plot loss function
fig = plt.figure()
plt.style.use('seaborn')

plt.plot(history_callback.history['loss'], 'c-', label='train')
plt.plot(history_callback.history['val_loss'], 'm:', label='test')
# plt.text(epoch-2, 0.07, rmse , style='italic')
plt.title(title)
plt.legend()
plt.grid(True)
fig.set_size_inches(w=7,h=7)
plt.close()
display(fig)

In [372]:
model = load_model('model_lstm_3.h5')

In [440]:
length = 1455

In [3]:
#creating a list of predictions 

test_predictions = []

first_eval_batch = scaled_train[-length:]

current_batch = first_eval_batch.reshape(1,length,n_features)


for i in range(len(test)):
  current_pred = model.predict(current_batch)[0]

  test_predictions.append(current_pred)

  current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis = 1)

In [442]:
#converting the predictions to a df to append to test dataset

true_predictions = scaler.inverse_transform(test_predictions)
true_predictions_df = pd.DataFrame(true_predictions)
col_names = list(test)
true_predictions_df.columns = col_names

In [4]:
#appending predictions to test dataset and plotting

eur_jpy = pd.DataFrame()
eur_jpy['EUR_JPY'] = test['eur_jpy']
eur_jpy['LSTM Predictions'] = true_predictions_df['eur_jpy'].values
eur_jpy.plot(figsize=(16,12))

In [447]:
eur_jpy.to_csv('model_predictions.csv')